In [131]:
import pandas as pd
import plotly.express as px
import pycountry

In [132]:
brand_df = pd.read_csv('../../data/all_brand_ratings.csv')

brand_df.head(5)

,brand,es_rating,es_transparency,es_fair_labor,es_sustainably_made,sr_rating,sr_description,goy_rating,goy_planet,goy_people,...,goy_description,goyr_rating,goyr_planet,goyr_people,goyr_animals,goyr_description,total_ratings,price_level,location,url
0,KNOWN SUPPLY,1.7,1.7,5.0,1.7,4.0,"<a data-wpel-link=""external"" href=""https://kno...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.850000,2.0,United States,https://directory.goodonyou.eco/brand/known-su...
1,AMENDI,1.7,5.0,3.3,1.7,4.0,AMENDI is a brand that is making commendable e...,NaN,NaN,NaN,...,NaN,4.0,4.0,3.0,4.0,AMENDI's environment rating is 'good'. It uses...,3.233333,3.0,Sweden,https://directory.goodonyou.eco/brand/amendi
2,SANVT,1.7,5.0,1.7,1.7,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.700000,NaN,NaN,NaN
3,Patagonia,5.0,5.0,5.0,5.0,4.0,Patagonia is a brand that focuses on sustainab...,4.0,4.0,2.0,...,Our “Planet” rating evaluates brands based on ...,4.0,4.0,2.0,4.0,Our “Planet” rating evaluates brands based on ...,4.250000,3.0,United States,https://directory.goodonyou.eco/brand/patagonia
4,Scotch & Soda,3.3,5.0,3.3,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.300000,2.0,Australia,https://directory.goodonyou.eco/brand/scotch-a...


In [133]:
country_counts = brand_df['location'].value_counts()
country_counts = pd.DataFrame(country_counts).reset_index()
country_counts.head()

,location,count
0,United States,368
1,United Kingdom,202
2,Australia,146
3,Germany,78
4,Canada,49


In [134]:
# convert country name to ISO Alpha-3 code
def get_iso3(country_name):
    try:
        return pycountry.countries.lookup(country_name).alpha_3
    except LookupError:
        return None

# add ISO Alpha-3 code column
country_counts['iso_alpha3'] = country_counts['location'].apply(get_iso3)

country_counts.head()

,location,count,iso_alpha3
0,United States,368,USA
1,United Kingdom,202,GBR
2,Australia,146,AUS
3,Germany,78,DEU
4,Canada,49,CAN


In [135]:
country_counts[country_counts['iso_alpha3'].isna()]

,location,count,iso_alpha3
24,Turkey,3,None
43,Swaziland,1,None
56,"Macedonia, the Former Yugoslav Republic of",1,None


In [136]:
# update countries that did not get a country code
manual_mapping = {
    'Turkey': 'TUR',
    'Swaziland': 'SWZ',
    'Macedonia, the Former Yugoslav Republic of': 'MKD'
}

# replace with ISO Alpha-3 codes from manual mapping
country_counts['iso_alpha3'] = country_counts.apply(lambda x: manual_mapping[x['location']] if x['iso_alpha3'] is None and x['location'] in manual_mapping else x['iso_alpha3'], axis=1)
country_counts.head()

,location,count,iso_alpha3
0,United States,368,USA
1,United Kingdom,202,GBR
2,Australia,146,AUS
3,Germany,78,DEU
4,Canada,49,CAN


In [137]:
# calculate the average price level for each country
average_price_level_by_country = brand_df.groupby('location')['price_level'].mean().reset_index()

country_counts = country_counts.merge(average_price_by_country, on='location', how='left')
country_counts = country_counts.rename(columns={'price_level': 'avg_price_level'})
country_counts.head()

,location,count,iso_alpha3,avg_price_level
0,United States,368,USA,2.410326
1,United Kingdom,202,GBR,2.554455
2,Australia,146,AUS,2.294521
3,Germany,78,DEU,2.333333
4,Canada,49,CAN,2.510204


In [138]:
# calculate the average brand rating for each country
average_brand_rating_by_country = brand_df.groupby('location')['total_ratings'].mean().reset_index()

country_counts = country_counts.merge(average_brand_rating_by_country, on='location', how='left')
country_counts = country_counts.rename(columns={'total_ratings':'avg_rating'})
country_counts.head()

,location,count,iso_alpha3,avg_price_level,avg_rating
0,United States,368,USA,2.410326,3.453442
1,United Kingdom,202,GBR,2.554455,3.831889
2,Australia,146,AUS,2.294521,3.919863
3,Germany,78,DEU,2.333333,3.941026
4,Canada,49,CAN,2.510204,3.732143


In [155]:
# create the choropleth map
# source: https://stackoverflow.com/questions/68017847/color-map-based-on-dataframe-with-countries-counts-values
map = px.choropleth(country_counts, 
                    locations="iso_alpha3",
                    color="count", 
                    hover_name="location",
                    color_continuous_scale=px.colors.sequential.Purpor,
                    labels={'count': 'Count'}
)

map.update_layout(
    coloraxis_colorbar=dict(title="Count"),
    title_text='Clothing Brand Counts by Country',
    title_x=0.5,
    width=800
)

map.show()

# save the visualization as interactive:
# import plotly.io as pio
# pio.write_html(map, file='interactive_plots/map_country_brand_count.html', auto_open=False)


In [154]:
# create the choropleth map
map = px.choropleth(country_counts, 
                    locations="iso_alpha3",
                    color="avg_price_level", 
                    hover_name="location",
                    color_continuous_scale=px.colors.sequential.RdPu
)

map.update_layout(
    coloraxis_colorbar=dict(title="Price Level"),
    title_text='Average Price Level by Country',
    title_x=0.5,
    width=800
)

map.show()

# note: need to save manually by clicking "Download plot as a png"

# save the visualization as interactive:
import plotly.io as pio
pio.write_html(map, file='interactive_plots/map_country_price_level.html', auto_open=False)


In [150]:
fig = px.scatter(country_counts, x='avg_price_level', y='avg_rating', size='count', color='location', hover_name='location',
                 labels={'avg_price_level': 'Avg Price Level', 'avg_rating': 'Average Sustainability Rating'})

fig.update_layout(title='Brand Count, Price Level, and Sustainability Rating by Country',
                  xaxis_title='Average Price Level',
                  yaxis_title='Average Sustainability Rating',
                  width=800
)

fig.show()

# note: need to save manually by clicking "Download plot as a png"

# save the visualization as interactive:
# import plotly.io as pio
# pio.write_html(fig, file='interactive_plots/bubble_chart.html', auto_open=False)
